In [24]:
import itertools
import numpy as np

In [43]:
def Gale_Shaplay(X, Y, n_iters=1000) -> np.ndarray:
    """ルールベースでのGale_Shaplayのアルゴリズム

    Args:
        X (_type_): マッチングを探す集合の希望行列, 値が大きいほど希望が高いとする
        Y (_type_): マッチングに応える集合の希望行列, 値が大きいほど希望が高いとする
        n_iters (int, optional): マッチングの試行回数. Defaults to 1000.

    Returns:
        np.ndarray: Xから見た相手となるYのインデックス
    """
    result_y = np.zeros(len(X)) - 1

    next_choice_for_xs = list(range(len(X) - 1, -1, -1))
    count = 0
    while len(next_choice_for_xs) > 0:
        # next_choice_for_xsの最後のxさんの希望の高い順にyさんたちを並べる
        x_id = next_choice_for_xs.pop()
        print(f"===x{x_id}さんのマッチングを開始します===")
        best_y_ranking_ids = np.argsort(X[x_id] * -1)
        for best_y_id in best_y_ranking_ids:
            print(f">希望の相手はy{best_y_id}さんです")
            # 希望のyさんが誰とも被っていなかった場合
            if np.isin(result_y, [best_y_id]).sum() == 0:
                print(f">>y{best_y_id}さんは誰とも希望が被らなかったのでマッチング成立です")
                # 被っていない人のidを取得
                result_y[x_id] = best_y_id
                break
            # 希望のyさんが誰か被っていた場合
            else:
                # yさんを選んだxのidを取得する
                rival_x_id = np.where(result_y == best_y_id)[0][0]
                print(f">>y{best_y_id}さんとマッチング済みのx{rival_x_id}さんがいました")
                # yさんの希望がxさんの方が高かった場合
                if Y[best_y_id][x_id] > Y[best_y_id][rival_x_id]:
                    print(f">>>y{best_y_id}さんはx{x_id}さんの方が希望が高かったため、マッチング相手を更新します")
                    # xさんの相手をyさんに更新
                    result_y[x_id] = best_y_id
                    # rival_xさんの相手を無しにする
                    result_y[rival_x_id] = -1
                    # rival_xさんをもう一度マッチング列の最前列に並ばせる
                    next_choice_for_xs.append(rival_x_id)
                    break
                # yさんの希望がrival_xさんの方が高かった場合
                else:
                    print(f">>>y{best_y_id}さんはx{rival_x_id}さんの方が希望が高かったため、マッチングは見送られました")
                    # xさんの次の希望を聞く
                    # xとyが同数の場合、必ず相手は見つかる
                    continue
        # iter_countを更新
        count += 1
        if count > n_iters:
            break
        print(f"現在のマッチング状況：{result_y}")

    return result_y


def brute_force(X, Y) -> dict[np.ndarray]:
    """総当たりでのマッチングアルゴリズム

    Args:
        X (_type_): マッチングを探す集合の希望行列, 値が大きいほど希望が高いとする
        Y (_type_): マッチングに応える集合の希望行列, 値が大きいほど希望が高いとする

    Returns:
        dict[np.ndarray]: 結果のdict, keyがマッチングのスコアでvalueがマッチングの組み合わせ
    """
    best_score = 0
    best_result = []
    permutation_X = itertools.permutations(range(len(X)))
    permutation_Y = itertools.permutations(range(len(Y)))

    # 総当たりでマッチングのスコアを計算する
    for xs in permutation_X:
        for ys in permutation_Y:
            score = 0
            for i in range(min(len(X), len(Y))):
                score += X[xs[i], ys[i]]
                score += Y[ys[i], xs[i]]
            if score >= best_score:
                result = [-1] * len(X)
                for j, x in enumerate(xs):
                    if j < len(ys):
                        result[x] = ys[j]
                    else:
                        break
                best_result.append(result)
                best_score = score

    return {int(best_score): best_result}

In [36]:
M = np.array(
    [[3, 2, 1, 0],
    [0, 3, 2, 1],
    [1, 0, 3, 2],
    [2, 1, 0, 3]]
)
W = np.array(
    [[3, 2, 1, 0],
    [0, 3, 2, 1],
    [1, 0, 3, 2],
    [2, 1, 0, 3]]
)

print(Gale_Shaplay(M, W))

===0さんのマッチングを開始します===
希望の相手は0さんです
0さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0. -1. -1. -1.]
===1さんのマッチングを開始します===
希望の相手は1さんです
1さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0.  1. -1. -1.]
===2さんのマッチングを開始します===
希望の相手は2さんです
2さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0.  1.  2. -1.]
===3さんのマッチングを開始します===
希望の相手は3さんです
3さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[0. 1. 2. 3.]
[0. 1. 2. 3.]


In [37]:
print(brute_force(M, W))

{24: [[0, 1, 2, 3]]}


In [44]:
M = np.array(
    [[3, 0, 1, 2],
    [3, 2, 1, 0],
    [2, 3, 1, 0],
    [0, 3, 2, 1]]
)
W = np.array(
    [[3, 2, 1, 0],
    [0, 3, 2, 1],
    [1, 0, 3, 2],
    [2, 1, 0, 3]]
)

print(Gale_Shaplay(M, W))

===x0さんのマッチングを開始します===
>希望の相手はy0さんです
>>y0さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0. -1. -1. -1.]
===x1さんのマッチングを開始します===
>希望の相手はy0さんです
>>y0さんとマッチング済みのx0さんがいました
>>>y0さんはx0さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy1さんです
>>y1さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0.  1. -1. -1.]
===x2さんのマッチングを開始します===
>希望の相手はy1さんです
>>y1さんとマッチング済みのx1さんがいました
>>>y1さんはx1さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy0さんです
>>y0さんとマッチング済みのx0さんがいました
>>>y0さんはx0さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy2さんです
>>y2さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0.  1.  2. -1.]
===x3さんのマッチングを開始します===
>希望の相手はy1さんです
>>y1さんとマッチング済みのx1さんがいました
>>>y1さんはx1さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy2さんです
>>y2さんとマッチング済みのx2さんがいました
>>>y2さんはx2さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy3さんです
>>y3さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[0. 1. 2. 3.]
[0. 1. 2. 3.]


In [45]:
print(brute_force(M, W))

{19: [[0, 1, 2, 3]]}


In [46]:
M = np.array(
    [[3, 0, 1, 2],
    [3, 2, 1, 0],
    [2, 3, 1, 0],
    ]
)
W = np.array(
    [[2, 1, 0],
    [0, 2, 1],
    [1, 0, 2],
    [2, 1, 0]]
)

print(Gale_Shaplay(M, W))

===x0さんのマッチングを開始します===
>希望の相手はy0さんです
>>y0さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0. -1. -1.]
===x1さんのマッチングを開始します===
>希望の相手はy0さんです
>>y0さんとマッチング済みのx0さんがいました
>>>y0さんはx0さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy1さんです
>>y1さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 0.  1. -1.]
===x2さんのマッチングを開始します===
>希望の相手はy1さんです
>>y1さんとマッチング済みのx1さんがいました
>>>y1さんはx1さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy0さんです
>>y0さんとマッチング済みのx0さんがいました
>>>y0さんはx0さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy2さんです
>>y2さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[0. 1. 2.]
[0. 1. 2.]


In [47]:
print(brute_force(M, W))

{12: [[0, 1, 2], [3, 0, 1]]}


In [ ]:
N = 5
M = np.random.randint(0, N, (N, N))
W = np.random.randint(0, N, (N, N))

print(Gale_Shaplay(M, W))

===x0さんのマッチングを開始します===
>希望の相手はy3さんです
>>y3さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
===x1さんのマッチングを開始します===
>希望の相手はy2さんです
>>y2さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3.  2. -1. -1. -1. -1. -1. -1. -1. -1.]
===x2さんのマッチングを開始します===
>希望の相手はy5さんです
>>y5さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3.  2.  5. -1. -1. -1. -1. -1. -1. -1.]
===x3さんのマッチングを開始します===
>希望の相手はy4さんです
>>y4さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3.  2.  5.  4. -1. -1. -1. -1. -1. -1.]
===x4さんのマッチングを開始します===
>希望の相手はy6さんです
>>y6さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3.  2.  5.  4.  6. -1. -1. -1. -1. -1.]
===x5さんのマッチングを開始します===
>希望の相手はy9さんです
>>y9さんは誰とも希望が被らなかったのでマッチング成立です
現在のマッチング状況：[ 3.  2.  5.  4.  6.  9. -1. -1. -1. -1.]
===x6さんのマッチングを開始します===
>希望の相手はy2さんです
>>y2さんとマッチング済みのx1さんがいました
>>>y2さんはx6さんの方が希望が高かったため、マッチング相手を更新します
現在のマッチング状況：[ 3. -1.  5.  4.  6.  9.  2. -1. -1. -1.]
===x1さんのマッチングを開始します===
>希望の相手はy2さんです
>>y2さんとマッチング済みのx6さんがいました
>>>y2さんはx6さんの方が希望が高かったため、マッチングは見送られました
>希望の相手はy0さんです
>>y0さ

In [49]:
print(brute_force(M, W))

{137: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 9, 8, 7], [0, 1, 2, 3, 4, 5, 7, 9, 8, 6], [0, 1, 2, 3, 4, 6, 5, 9, 8, 7], [0, 1, 2, 3, 4, 6, 9, 5, 8, 7], [0, 1, 2, 3, 5, 6, 9, 4, 8, 7], [0, 1, 2, 4, 5, 6, 7, 9, 8, 3], [0, 1, 3, 2, 5, 6, 9, 4, 8, 7], [0, 1, 3, 4, 5, 6, 2, 9, 8, 7], [0, 1, 3, 4, 5, 6, 7, 9, 8, 2], [0, 1, 5, 2, 3, 6, 9, 4, 8, 7], [0, 1, 5, 2, 4, 6, 7, 9, 8, 3], [0, 1, 5, 3, 4, 6, 2, 9, 8, 7], [0, 1, 5, 3, 4, 6, 7, 9, 8, 2], [0, 2, 3, 4, 5, 6, 9, 1, 8, 7], [0, 2, 5, 1, 3, 6, 9, 4, 8, 7], [0, 2, 5, 1, 4, 6, 7, 9, 8, 3], [0, 2, 5, 3, 1, 6, 9, 4, 8, 7], [0, 2, 5, 3, 4, 6, 1, 9, 8, 7], [0, 2, 5, 3, 4, 6, 9, 1, 8, 7], [1, 0, 5, 3, 4, 6, 2, 9, 8, 7], [1, 0, 5, 3, 4, 6, 7, 9, 8, 2], [2, 0, 5, 3, 1, 6, 9, 4, 8, 7], [2, 0, 5, 3, 4, 6, 1, 9, 8, 7], [2, 0, 5, 3, 4, 6, 9, 1, 8, 7], [4, 0, 5, 1, 3, 6, 2, 9, 8, 7], [4, 0, 5, 1, 3, 6, 7, 9, 8, 2], [4, 0, 5, 3, 1, 6, 2, 9, 8, 7], [4, 0, 5, 3, 1, 6, 7, 9, 8, 2]]}


In [50]:
M

array([[2, 1, 5, 9, 6, 6, 2, 1, 3, 7],
       [8, 1, 9, 7, 6, 2, 0, 4, 3, 6],
       [2, 7, 2, 2, 3, 9, 5, 5, 6, 3],
       [3, 4, 4, 7, 9, 0, 6, 1, 1, 4],
       [2, 3, 1, 5, 5, 5, 8, 2, 8, 5],
       [3, 5, 1, 3, 4, 2, 7, 2, 4, 8],
       [5, 6, 9, 2, 9, 3, 5, 6, 2, 9],
       [4, 8, 4, 2, 9, 1, 0, 6, 3, 9],
       [3, 3, 3, 7, 2, 2, 7, 0, 9, 2],
       [0, 1, 8, 5, 2, 1, 5, 4, 4, 4]])